In [1]:
import os, json, numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import precision_recall_fscore_support

In [2]:
# -----------------------
# Paths & hyperparams
# -----------------------

def find_project_root() -> str:
    """
    Try current dir and up to 4 parents to locate a folder that contains 'backend/ml/dataset/raw'.
    If not found, return current working directory.
    """
    cwd = os.getcwd()
    candidates = [cwd]
    # try parents
    cur = cwd
    for _ in range(4):
        cur = os.path.dirname(cur)
        if cur and cur not in candidates:
            candidates.append(cur)
    for base in candidates:
        raw_dir = os.path.join(base, "backend", "ml", "dataset", "raw")
        if os.path.isdir(raw_dir):
            return base
    return cwd

PROJECT_ROOT = find_project_root()
DATA_DIR = os.path.join(PROJECT_ROOT, "backend", "ml", "dataset", "processed")
MODEL_DIR = os.path.join(PROJECT_ROOT, "backend", "ml", "models")
EVAL_DIR  = os.path.join(PROJECT_ROOT, "backend", "ml", "evaluation")
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(EVAL_DIR, exist_ok=True)

DEVICE = torch.device("cpu")
BATCH_SIZE = 64
EPOCHS = 50
PATIENCE = 5
LR = 1e-3
D_MODEL = 64        # Transformer hidden dimension
NHEAD = 4
NUM_LAYERS = 2
FF_DIM = 128

In [3]:
# -----------------------
# Load processed arrays
# -----------------------
def must(path):
    if not os.path.exists(path):
        raise FileNotFoundError(
            f"Missing: {path}\nRun preprocessing first to generate npy files."
        )
    return path

X_train = np.load(must(os.path.join(DATA_DIR, "X_train.npy"))).astype(np.float32)
y_train = np.load(must(os.path.join(DATA_DIR, "y_train.npy"))).astype(np.int64)
X_val   = np.load(must(os.path.join(DATA_DIR, "X_val.npy"))).astype(np.float32)
y_val   = np.load(must(os.path.join(DATA_DIR, "y_val.npy"))).astype(np.int64)
X_test  = np.load(must(os.path.join(DATA_DIR, "X_test.npy"))).astype(np.float32)
y_test  = np.load(must(os.path.join(DATA_DIR, "y_test.npy"))).astype(np.int64)

cfg_path = os.path.join(DATA_DIR, "feature_config.json")
with open(cfg_path, "r") as f:
    feat_cfg = json.load(f)

SEQ_LEN = int(X_train.shape[1])
INPUT_SIZE = int(X_train.shape[2])
NUM_CLASSES = int(max(y_train.max(), y_val.max(), y_test.max()) + 1)

print(f"Shapes X: {X_train.shape}, y: {y_train.shape}")
print(f"SEQ_LEN={SEQ_LEN}, INPUT_SIZE={INPUT_SIZE}, CLASSES={NUM_CLASSES}")

Shapes X: (6241, 32, 3), y: (6241, 32)
SEQ_LEN=32, INPUT_SIZE=3, CLASSES=2


In [4]:
# -----------------------
# DataLoaders
# -----------------------
Xtr_t, ytr_t = torch.tensor(X_train), torch.tensor(y_train)
Xva_t, yva_t = torch.tensor(X_val),   torch.tensor(y_val)
Xte_t, yte_t = torch.tensor(X_test),  torch.tensor(y_test)

train_loader = DataLoader(TensorDataset(Xtr_t, ytr_t), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(TensorDataset(Xva_t, yva_t), batch_size=BATCH_SIZE)
test_loader  = DataLoader(TensorDataset(Xte_t, yte_t), batch_size=BATCH_SIZE)

# -----------------------
# Model: Transformer Encoder for per-timestep classification
# -----------------------
class TransformerEncoderModel(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_layers, ff_dim, num_classes):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=ff_dim,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.input_proj(x)         # [B, T, d_model]
        x = self.encoder(x)            # [B, T, d_model]
        logits = self.classifier(x)    # [B, T, num_classes]
        return logits

model = TransformerEncoderModel(
    input_dim=INPUT_SIZE,
    d_model=D_MODEL,
    nhead=NHEAD,
    num_layers=NUM_LAYERS,
    ff_dim=FF_DIM,
    num_classes=NUM_CLASSES
).to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [5]:
# -----------------------
# Training Loop
# -----------------------
best_val = float("inf")
patience = 0
best_model_path = os.path.join(MODEL_DIR, "transformer_model.pt")

for epoch in range(EPOCHS):
    model.train()
    tr_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        optimizer.zero_grad()
        logits = model(xb)  # [B,T,C]
        loss = criterion(logits.view(-1, NUM_CLASSES), yb.view(-1))
        loss.backward()
        optimizer.step()
        tr_loss += loss.item() * xb.size(0)
    tr_loss /= len(train_loader.dataset)

    # Validation
    model.eval()
    va_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            logits = model(xb)
            loss = criterion(logits.view(-1, NUM_CLASSES), yb.view(-1))
            va_loss += loss.item() * xb.size(0)
    va_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch+1:02d} | Train {tr_loss:.4f} | Val {va_loss:.4f}")

    if va_loss < best_val:
        best_val = va_loss
        patience = 0
        torch.save(model.state_dict(), best_model_path)
    else:
        patience += 1
        if patience >= PATIENCE:
            print("Early stopping.")
            break

Epoch 01 | Train 0.6560 | Val 0.5941
Epoch 02 | Train 0.5602 | Val 0.5659
Epoch 03 | Train 0.4936 | Val 0.3938
Epoch 04 | Train 0.4028 | Val 0.3323
Epoch 05 | Train 0.3708 | Val 0.2980
Epoch 06 | Train 0.3478 | Val 0.3474
Epoch 07 | Train 0.3247 | Val 0.2408
Epoch 08 | Train 0.2965 | Val 0.2416
Epoch 09 | Train 0.3106 | Val 0.3420
Epoch 10 | Train 0.2979 | Val 0.2307
Epoch 11 | Train 0.2839 | Val 0.1776
Epoch 12 | Train 0.2770 | Val 0.3386
Epoch 13 | Train 0.2459 | Val 0.3609
Epoch 14 | Train 0.3063 | Val 0.2400
Epoch 15 | Train 0.2560 | Val 0.2045
Epoch 16 | Train 0.2426 | Val 0.1968
Early stopping.


In [6]:
# -----------------------
# Evaluation
# -----------------------
model.load_state_dict(torch.load(best_model_path, map_location=DEVICE))
model.eval()

all_preds, all_tgts = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(DEVICE)
        logits = model(xb)
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        all_preds.append(preds)
        all_tgts.append(yb.numpy())

all_preds = np.concatenate(all_preds, axis=0).reshape(-1)
all_tgts = np.concatenate(all_tgts, axis=0).reshape(-1)

precision, recall, f1, _ = precision_recall_fscore_support(all_tgts, all_preds, average='binary')
metrics = {"precision": float(precision), "recall": float(recall), "f1_score": float(f1)}

with open(os.path.join(EVAL_DIR, "transformer_metrics.json"), "w") as f:
    json.dump(metrics, f, indent=2)
print("Test metrics:", metrics)

Test metrics: {'precision': 0.9188186749501599, 'recall': 0.9230554261760596, 'f1_score': 0.9209321777922349}


In [7]:
# -----------------------
# Export to ONNX
# -----------------------
dummy = torch.randn(1, SEQ_LEN, INPUT_SIZE, device=DEVICE)
onnx_path = os.path.join(MODEL_DIR, "transformer_model.onnx")

try:
    import onnx
except ImportError:
    raise RuntimeError("Missing ONNX. Install with: pip install onnx")

torch.onnx.export(
    model, dummy, onnx_path,
    input_names=["input"], output_names=["logits"],
    dynamic_axes={"input": {0:"batch", 1:"seq_len"}, "logits": {0:"batch", 1:"seq_len"}},
    opset_version=14
)
print("Exported ONNX model to:", onnx_path)

# Save feature config for inference
with open(os.path.join(EVAL_DIR, "transformer_feature_config.json"), "w") as f:
    json.dump({
        "seq_len": SEQ_LEN,
        "features": feat_cfg.get("features", [f"f{i}" for i in range(INPUT_SIZE)])
    }, f, indent=2)
print("Saved transformer feature config.")

C:\Users\geloq\AppData\Local\Temp\ipykernel_3864\3160647740.py:12: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


Exported ONNX model to: c:\Users\geloq\OneDrive\Desktop\pd-keyboard-app\backend\ml\models\transformer_model.onnx
Saved transformer feature config.
